# permits-data / Load Data

ETL pipeline for construction permits data in Los Angeles, California, USA.

For more information:
https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf

In [1]:
import os
import sys
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd
import psycopg2

In [2]:
# Set notebook display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# Get project root directory
root_dir = os.path.dirname(os.getcwd())

# Set path for modules
sys.path[0] = '../'

# Set environment variables
load_dotenv(find_dotenv());
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")
DB_PORT = os.getenv("DB_PORT")
DB_HOST = os.getenv("DB_HOST")
DATA_URL = os.getenv("DATA_URL")

# Environment variables specific to notebook
DATA_DIR = os.path.dirname(root_dir) + '/data'
DB_TABLE = "permits_raw"

## 1. Import Data

In [4]:
# Connect to PostgreSQL, useful only for notebook
def connect_db():
    try:
        con = psycopg2.connect(dbname=POSTGRES_DB,
                               user=POSTGRES_USER,
                               password=POSTGRES_PASSWORD,
                                host=DB_HOST, 
                                port=DB_PORT,
                              connect_timeout=3)
        print('Connected as user "{}" to database "{}" on {}:{}\n'.format(POSTGRES_USER,POSTGRES_DB,
                                                           DB_HOST,DB_PORT))
              
    except Exception as e:
        print('Error:\n', e)
    
    return con

In [5]:
conn = connect_db()

Connected as user "postgres" to database "permits" on localhost:5432



### 1.1 Update Table Columns in PostgreSQL Database

In [6]:
# Get raw data column names
def get_table_names(table, con):
    sql = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{}'".format(table)
    etl = pd.read_sql_query(sql, con)
    columns = etl['column_name']
    
    return columns

In [7]:
# Check table names
get_table_names("permits_raw", conn).head(10)

0                 Assessor Book
1                 Assessor Page
2               Assessor Parcel
3                         Tract
4                         Block
5                           Lot
6    Reference # (Old Permit #)
7                 PCIS Permit #
8                        Status
9                   Status Date
Name: column_name, dtype: object

In [8]:
# Retrieve table column names
old_columns = get_table_names("permits_raw", conn)

In [9]:
# Rename columns, will update table later
def format_names(series):
    
    replace_map = {' ': '_', '-': '_', '#': 'No', '/': '_', 
                   '.': '', '(': '', ')': '', "'": ''}

    def replace_chars(text):
        for oldchar, newchar in replace_map.items():
            text = text.replace(oldchar, newchar).lower()
        return text

    return series.apply(replace_chars)

In [10]:
# Transform table column names for permits_raw
new_columns = format_names(old_columns)

In [11]:
new_columns.head()

0      assessor_book
1      assessor_page
2    assessor_parcel
3              tract
4              block
Name: column_name, dtype: object

In [12]:
# Creates a SQL query to update table columns and writes to text file
### add path string
def create_query(old_columns, new_columns, db_table, con, run=False):
    
    sql = 'ALTER TABLE {} '.format(db_table) + 'RENAME "{old_name}" to {new_name};'
    
    
    sql_query = []

    for idx, name in old_columns.iteritems():
        sql_query.append(sql.format(old_name=name, new_name=new_columns[idx]))
        
    update_names = '\n'.join(sql_query)
    
    # replace with path
    with open('../postgres/sql/update_names.sql', 'w') as text:
        text.write(update_names)
        
    # Update db is desired
    if run:
        # try in case connection not open
        cur = con.cursor()
        # replace with path
        sql_file = open('../postgres/sql/update_names.sql', 'r')
        cur.execute(sql_file.read())
        con.commit()
        #conn.close()

In [13]:
# Create SQL query for permits_raw
try:
    create_query(old_columns, new_columns, run=True, con=conn, db_table=DB_TABLE)
except Exception as e: 
    conn.rollback()
    print('Error:\n', e)

In [14]:
# Check table names are updated
get_table_names("permits_raw", conn).head()

0      assessor_book
1      assessor_page
2    assessor_parcel
3              tract
4              block
Name: column_name, dtype: object

In [15]:
# TEST: 
assert (get_table_names("permits_raw", 
                        conn) == new_columns).mean() == 1, "Database table names do not match new table names"

In [16]:
# Extract full dataset
sql_all = 'SELECT * FROM {} LIMIT 1500;'.format(DB_TABLE)

# Extract full dataset
data = pd.read_sql_query(sql_all, conn)
data.head()

,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code
0,4317,003,***,TR 30210-C,None,LT 1,None,15044-90000-08405,Permit Finaled,09/10/2015,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2015-08-18,1823,1/2,1823,1/2,S,THAYER,AVE,None,None,None,90025,None,NaN,None,None,None,None,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,2016-06-30,BRETT,HOFFER,None,18650 PARTHENIA ST,None,"NORTHRIDGE, CA",R3-1-O,None,0,2671.00,5,"(34.05474, -118.42628)",Net Applicant,None,None
1,5005,010,017,CHESTERFIELD SQUARE,None,465,16SL57806,16016-70000-02464,Permit Finaled,08/01/2017,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,None,None,SOUTH LA,2016-02-04,2122,None,2122,None,W,54TH,ST,None,None,None,90062,General rehabilitation for single family dwell...,40000.0,None,None,None,None,OWNER-BUILDER,None,None,None,NA,0,JAVIER,None,TALAMANTES,None,JAVIER,TALAMANTES,OWNER-BUILDER,None,None,None,C2-1VL,None,None,2325.00,8,"(33.99307, -118.31668)",Owner-Bldr,1,None
2,5154,023,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,08/13/2014,Bldg-Alter/Repair,Apartment,Plan Check,None,None,VAN NUYS,2014-08-13,415,None,415,None,S,BURLINGTON,AVE,None,1-30,1-30,90057,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.0,None,None,None,None,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,2015-11-30,LINDA,MARTON,None,710 WILSHIRE BLVD,None,"SANTA MONICA, CA",R4-1,None,None,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5,None
3,4404,030,010,TR 12086,None,2,None,16044-30000-09658,Permit Finaled,08/29/2016,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,WEST LA,2016-08-22,315,None,315,None,S,OCEANO,DR,None,None,None,90049,None,NaN,None,None,None,None,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,2018-07-31,None,None,None,None,None,None,RS-1,None,0,2640.00,11,"(34.05707, -118.4732)",Contractor,None,None
4,2646,019,011,TR 7158,None,11,None,17042-90000-31792,Permit Finaled,12/28/2017,Plumbing,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2017-12-26,13640,None,13640,None,W,PIERCE,ST,None,None,None,91331,None,NaN,None,None,None,None,TITANIUM POWER INC,1545 S LA CIENEGA BLVD,LOS ANGELES,CA,B,989217,DENNIS,HARUO,MIYAHIRA,2017-12-31,YONI,GHERMEZI,None,1545 S LA CIENEGA BLVD,None,"LOS ANGELES, CA",R1-1-O,None,0,1044.03,7,"(34.25487, -118.43002)",Net Applicant,None,None


### 1.2 Update Data Types in database

Before working with the data it will be important to have the correct data types mapped to the values. That way when extracting and loading into a Pandas dataframe, for example, it is immediately ready for analysis. 

Determining the correct data types involves some preliminary exploration of the data for which uses two helper functions: *get_snapshot* and 

In [17]:
### Returns overview with column, dtype, # unique values, # missing values and sample value
def get_snapshot(dataframe):
    
    """
    Takes an existing DataFrame and creates a pandas DataFrame 
    where each row displays the original DataFrame column name, 
    number of unique values, number of missing values, and a
    random sample value from that column. 
    
    Useful for exploring raw data to quickly figure out appropriate 
    data types.
    
    Example
    -------
    
    overview = get_overview(my_dataframe)
    
    """
    unique = dataframe.nunique(axis=0)
    is_null = dataframe.isnull().sum()
    data_types = dataframe.dtypes
    
    samples = pd.DataFrame()
    column_names = pd.DataFrame()
    
    for column, row in dataframe.iteritems():
        try:
            sample = dataframe[column].dropna(axis=0).sample()
            column_name = pd.Series(column)
        except:
            pass

        samples = pd.concat([samples, sample], axis=0).reset_index(drop=True)
        column_names = pd.concat([column_names, column_name], 
                                 axis=0, ).reset_index(drop=True)

        examples = pd.concat([column_names, samples], axis=1, ignore_index=True)
        examples.columns = ['COLUMN', 'SAMPLE VALUE']
    
    overview = pd.concat([data_types, unique, is_null], axis=1)
    overview.reset_index(inplace=True)
    overview.columns = columns=['COLUMN', 'DATA TYPE', '# UNIQUE VALUES', '# MISSING VALUES']
    overview = overview.merge(right=examples, on='COLUMN').drop_duplicates(subset=['COLUMN']).set_index('COLUMN')
    
    return overview

In [18]:
get_snapshot(data)

,DATA TYPE,# UNIQUE VALUES,# MISSING VALUES,SAMPLE VALUE
COLUMN,,,,
assessor_book,object,740,1,7563
assessor_page,object,49,1,006
assessor_parcel,object,111,1,114
tract,object,1202,5,TR 11309
block,object,96,1179,39
lot,object,296,14,83
reference_no_old_permit_no,object,520,877,13LA
pcis_permit_no,object,1500,0,12010-10002-00842
status,object,11,0,Permit Finaled


In [19]:
# Creates a report to show value counts for columns with less than n unique values
def explore_value_counts(dataframe, n=None, all_values=False, max_n=1500, columns=None, printed=True):
    
    """
    This function is helpful for quickly determining which values
    should be converted to integer or category types in a dataframe.
    
    Prints a series of custom text summaries with n value counts 
    for each column. Can work if neither n nor columns are specified.
    
    Also can return a generator yielding a text summary with n
    value counts for each column.
    
    Example:
    --------
    ## Iterates through individual tables
    gen = explore_value_counts(data, printed=False)
    print(next(gen)) 
    
    ## Prints all tables to STDOUT
    explore_value_counts(data, printed=True)
    
    Params
    --------
    dataframe : pandas DataFrame
        DataFrame with columns to be summarized.
    n : integer
        Max number of unique categories in column.
    all_values : bool
        Set to True to return all unique values. Will override n.
    max_n : integer
        Ceiling safeguard to avoid extremely large values of n
    columns : list 
        Columns to include in output.
    printed : bool
        If True prints to console; if False returns generator object
        which can be printed as text.
    
    Returns
    --------
    if printed=True: prints all formatted text of all tables
    
    if printed=False: generator object that outputs one table
    
    """
    
    # Parsing arguments
    if columns:
        dataframe = dataframe[columns]
    
    if n:
        n = n if n <= max_n else max_n
    else:
        n = len(dataframe) if len(dataframe) <= max_n else max_n
    
    if all_values:            
        n = len(dataframe) if len(dataframe) <= max_n else max_n
    
    if not n:
        n = 30
    
    def make_tables():
        dataframe_n = pd.DataFrame()

        # Data selection
        for column, row in dataframe.iteritems():
            
            n_unique = dataframe[column].nunique()
            
            # Throws error if float64 is removed
            if (dataframe[column].dtype not in ['float64', '<M8[ns]']):
                dataframe_n = pd.concat([dataframe_n, dataframe[column]], axis=1)

        summary_list = []

        # Text generation
        for column, row in dataframe_n.iteritems(): 
            series = dataframe_n[column]
            name = series.name
            
            index_slice = n if n <= len(series) else len(series)
            
            # Create dataframe of value counts
            counted = series.value_counts(sort=True)[:index_slice]
            percent = series.value_counts(sort=True, normalize=True)[:index_slice]
            summary = pd.concat([counted, percent], axis=1)
            summary.columns = ['COUNT', 'PERCENTAGE']
            summary.index = summary.index.rename('UNIQUE VALUES:')
            
            # Create a custom table with n unique, missing values to print to console as text
            summary_text = 'COLUMN:   "{}"\nDATA TYPE:  {}'.format(name, series.dtype)
            summary_text = summary_text + '\nTOTAL UNIQUE:  {}'.format(name, series.nunique())
            summary_text = summary_text + '\nTOTAL MISSING:  {}'.format(series.isnull().sum())
            summary_text = summary_text + '\n' + summary.to_string() + '\n\n'

            summary_list.append(summary_text)
        
        if not printed:
            summary_gen = iter(summary_list)
            return summary_gen
        else:
            return summary_list
        
    if printed:
        print('\n'.join(make_tables()))
    else:
        return make_tables()

In [30]:
# Examine variables to determine appropriate data types
explore_value_counts(data, columns=['block'], all_values=True, printed=True)

COLUMN:   "block"
DATA TYPE:  object
TOTAL UNIQUE:  block
TOTAL MISSING:  1179
                COUNT  PERCENTAGE
UNIQUE VALUES:                   
2                  20    0.062305
3                  18    0.056075
1                  14    0.043614
5                  13    0.040498
4                  12    0.037383
6                  11    0.034268
7                  10    0.031153
B                   9    0.028037
C                   9    0.028037
11                  9    0.028037
10                  8    0.024922
14                  8    0.024922
9                   7    0.021807
D                   7    0.021807
A                   7    0.021807
21                  6    0.018692
8                   6    0.018692
K                   5    0.015576
13                  5    0.015576
BLK B               5    0.015576
28                  4    0.012461
17                  4    0.012461
H                   4    0.012461
20                  4    0.012461
BLK 3               4    0.012461
15 

#### Summary of Data Types
The following columns will have their data types programmatically updated in the database:

***Pandas Category columns:***
* status - VARCHAR(30)
* permit_type - VARCHAR(30)
* permit_sub_type - VARCHAR(30)
* permit_category - VARCHAR(30)
* initiating_office - VARCHAR(30)
* license_type - VARCHAR(10)
* zone - VARCHAR(10)
* census_tract - VARCHAR(10)
* applicant_relationship - VARCHAR(30)

***Pandas Object columns:***
* 

***Pandas Integer columns:***
* council_district - SMALLINT
* project_number - SMALLINT
* address_start - INTEGER
* address_end - INTEGER
* no_of_residential_dwelling_units - SMALLINT
* no_of_accessory_dwelling_units - SMALLINT
* no_of_stories - SMALLINT
* license_no - INTEGER

***Pandas Float columns:***
* valuation - NUMERIC(12, 2)


In [21]:
# Updates column types in PostgreSQL database
def update_table_types(column_dict, sql_string, table, printed=False, 
                       write=False, path=None, run=False, con=None):
    
    """
    Takes a sql statement to ALTER COLUMN types (string)
    and appends it to an ALTER TABLE statement and prints
    or returns a full string. Made for PostgreSQL.
    
    Example
    ---------
    # Dictionary of SQL types
    numeric_cols = {'valuation': 'NUMERIC(12, 2)'}
    
    # ALTER column statement as string, do not end with ',' or ';'
    sql_numeric = "ALTER {column} TYPE {col_type} USING {column}::numeric"
    
    # Writes a out text file to disk
    update_table_types(integer_cols, sql_integer, printed=True, 
                                        write=True, path='./sql')
    
    Output
    ---------
    >>> "ALTER TABLE public.permits_raw
            ALTER valuation TYPE NUMERIC(12, 2) USING valuation::numeric;"
            
    Parameters
    ----------
    column_dict : dictionary
        Dictionary in form {'column name': 'SQL datatype'}
    
    sql_string : string
        Must be a SQL query string in form:
        "ALTER {column} TYPE {col_type} USING {column}::numeric
    
    printed : boolean
        If True will output to console
    
    write : boolean
        If True will write to specified path
    
    path : string
        Path of directory to write text file to
        
    run : boolean
        If True, will run query in database.
        
    con : psycopg2 connection object
        Required if run = True
    """
    
    # Define SQL update queries
    sql_alter_table = "ALTER TABLE public.{db_table}\n\t".format(db_table=table)

    # Append comma, new line and tab
    sql_string = sql_string + ",\n\t"
    
    # Update types
    sql_update_type = []
    for column, col_type in column_dict.items():
        sql_update_type.append(sql_string.format(column=column, col_type=col_type))
    
    # Join strings to create full sql query
    sql_update_type = sql_alter_table + ''.join(sql_update_type)
    
    # Replace very last character with ";"
    sql_update_type = sql_update_type[:-3] + ";"
    
    if printed:
        print(sql_update_type)
    
    if write:
        with open(path, 'w') as text:
            text.write(sql_update_type)
        print("\nSQL written to:\n{}\n".format(path))
    
        if run:
            #assert con, "No connection to database."
            # try in case connection not open
            try:
                print("Connecting...")
                cur = con.cursor()
                sql_file = open(path, 'r')
                print("Executing query...")
                cur.execute(sql_file.read())
                print("Committing changes...")
                con.commit()
                con.close()
                print("Database updated successfully.")
            except Exception as e:
                conn.rollback()
                print('Error:\n', e)
                
    elif run and not write:
        print('Set "write=True" and define path to run query from file.')
        
    return sql_update_type

#### Update VARCHAR types

In [22]:
conn = connect_db()

# Dictionary of columns and new varchar types
varchar_cols = {'status':'VARCHAR(50)', 'permit_type':'VARCHAR(50)', 'permit_sub_type':'VARCHAR(50)', 
                'permit_category':'VARCHAR(50)', 'initiating_office':'VARCHAR(50)', 
                'license_type':'VARCHAR(50)', 'zone':'VARCHAR(50)', 'census_tract':'VARCHAR(50)', 
                'applicant_relationship':'VARCHAR(50)'}

sql_varchar = "ALTER {column} TYPE {col_type}"

# Path to varchar query file
path_varchar = root_dir + '/postgres/sql/update_varchar.sql'

# Update column types
update_table_types(varchar_cols, sql_varchar, table='permits_raw', printed=True, 
                   write=True, path=path_varchar, run=True, con=conn);

Connected as user "postgres" to database "permits" on localhost:5432

ALTER TABLE public.permits_raw
	ALTER initiating_office TYPE VARCHAR(50),
	ALTER census_tract TYPE VARCHAR(50),
	ALTER permit_category TYPE VARCHAR(50),
	ALTER zone TYPE VARCHAR(50),
	ALTER permit_type TYPE VARCHAR(50),
	ALTER license_type TYPE VARCHAR(50),
	ALTER applicant_relationship TYPE VARCHAR(50),
	ALTER permit_sub_type TYPE VARCHAR(50),
	ALTER status TYPE VARCHAR(50);

SQL written to:
/Users/gregory/Documents/00 Data Projects/project-portfolio/permits-data/postgres/sql/update_varchar.sql

Connecting...
Executing query...
Committing changes...
Database updated successfully.


#### Update INTEGER types

In [27]:
conn = connect_db()

# Dictionary of columns and new integer types
integer_cols = {'council_district': 'SMALLINT', 'project_number': 'SMALLINT', 'address_start': 'INTEGER', 
                'address_end': 'INTEGER', 'no_of_residential_dwelling_units': 'SMALLINT', 
                'no_of_accessory_dwelling_units': 'SMALLINT', 'no_of_stories': 'SMALLINT', 
                'license_no': 'INTEGER'}

sql_integer = "ALTER {column} TYPE {col_type} USING {column}::{col_type}"

# Path to varchar query file
path_integer = root_dir + '/postgres/sql/update_integer.sql'

# Update column types
update_table_types(integer_cols, sql_integer, table='permits_raw', printed=True, 
                   write=True, path=path_integer, run=True, con=conn);

Connected as user "postgres" to database "permits" on localhost:5432

ALTER TABLE public.permits_raw
	ALTER address_start TYPE INTEGER USING address_start::INTEGER,
	ALTER license_no TYPE INTEGER USING license_no::INTEGER,
	ALTER council_district TYPE SMALLINT USING council_district::SMALLINT,
	ALTER address_end TYPE INTEGER USING address_end::INTEGER,
	ALTER no_of_accessory_dwelling_units TYPE SMALLINT USING no_of_accessory_dwelling_units::SMALLINT,
	ALTER project_number TYPE SMALLINT USING project_number::SMALLINT,
	ALTER no_of_stories TYPE SMALLINT USING no_of_stories::SMALLINT,
	ALTER no_of_residential_dwelling_units TYPE SMALLINT USING no_of_residential_dwelling_units::SMALLINT;

SQL written to:
/Users/gregory/Documents/00 Data Projects/project-portfolio/permits-data/postgres/sql/update_integer.sql

Connecting...
Executing query...
Committing changes...
Database updated successfully.


#### Update NUMERIC types

In [26]:
conn = connect_db()

# Dictionary of columns and new numeric types
numeric_cols = {'valuation': 'NUMERIC(12, 2)'}

sql_numeric = "ALTER {column} TYPE {col_type}"

# Path to varchar query file
path_numeric = root_dir + '/postgres/sql/update_numeric.sql'

# Update column types
update_table_types(numeric_cols, sql_numeric, table='permits_raw', printed=True, 
                   write=True, path=path_numeric, run=True, con=conn);

Connected as user "postgres" to database "permits" on localhost:5432

ALTER TABLE public.permits_raw
	ALTER valuation TYPE NUMERIC(12, 2);

SQL written to:
/Users/gregory/Documents/00 Data Projects/project-portfolio/permits-data/postgres/sql/update_numeric.sql

Connecting...
Executing query...
Committing changes...
Database updated successfully.


## 2. Clean Data

In [21]:
# Connect to db
conn = connect_db()

# Extract partial dataset
sql_all = 'SELECT * FROM {} LIMIT 1500;'.format(DB_TABLE)

# Columns to parse as dates
date_columns = ['status_date', 'issue_date', 'license_expiration_date']

# Fetch fresh data
data = pd.read_sql_query(sql_all, conn, parse_dates=date_columns, 
                         coerce_float=False)

Connected as user "postgres" to database "permits" on localhost:5432


In [22]:
data.head()

,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code
0,4317,003,***,TR 30210-C,None,LT 1,None,15044-90000-08405,Permit Finaled,2015-09-10,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2015-08-18,1823,1/2,1823,1/2,S,THAYER,AVE,None,None,None,90025.0,None,None,None,None,None,None,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,2016-06-30,BRETT,HOFFER,None,18650 PARTHENIA ST,None,"NORTHRIDGE, CA",R3-1-O,None,0,2671.00,5,"(34.05474, -118.42628)",Net Applicant,NaN,NaN
1,5005,010,017,CHESTERFIELD SQUARE,None,465,16SL57806,16016-70000-02464,Permit Finaled,2017-08-01,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,None,None,SOUTH LA,2016-02-04,2122,None,2122,None,W,54TH,ST,None,None,None,90062.0,General rehabilitation for single family dwell...,40000.00,None,None,None,None,OWNER-BUILDER,None,None,None,NA,0,JAVIER,None,TALAMANTES,NaT,JAVIER,TALAMANTES,OWNER-BUILDER,None,None,None,C2-1VL,None,None,2325.00,8,"(33.99307, -118.31668)",Owner-Bldr,1.0,NaN
2,5154,023,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,2014-08-13,Bldg-Alter/Repair,Apartment,Plan Check,None,None,VAN NUYS,2014-08-13,415,None,415,None,S,BURLINGTON,AVE,None,1-30,1-30,90057.0,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.00,None,None,None,None,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,2015-11-30,LINDA,MARTON,None,710 WILSHIRE BLVD,None,"SANTA MONICA, CA",R4-1,None,None,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5.0,NaN
3,4404,030,010,TR 12086,None,2,None,16044-30000-09658,Permit Finaled,2016-08-29,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,WEST LA,2016-08-22,315,None,315,None,S,OCEANO,DR,None,None,None,90049.0,None,None,None,None,None,None,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,2018-07-31,None,None,None,None,None,None,RS-1,None,0,2640.00,11,"(34.05707, -118.4732)",Contractor,NaN,NaN
4,2646,019,011,TR 7158,None,11,None,17042-90000-31792,Permit Finaled,2017-12-28,Plumbing,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2017-12-26,13640,None,13640,None,W,PIERCE,ST,None,None,None,91331.0,None,None,None,None,None,None,TITANIUM POWER INC,1545 S LA CIENEGA BLVD,LOS ANGELES,CA,B,989217,DENNIS,HARUO,MIYAHIRA,2017-12-31,YONI,GHERMEZI,None,1545 S LA CIENEGA BLVD,None,"LOS ANGELES, CA",R1-1-O,None,0,1044.03,7,"(34.25487, -118.43002)",Net Applicant,NaN,NaN


### 2.1 Exploring Missing Data & Data Types

#### Overview of Unique Values in Qualitative Data

Before making decisions regarding missing values or data types, it is important to be familiar with the content of each column especially considering the permits dataset mostly contains qualitative data.

#### Summary
* *zip_code* and *latitude_longitude* need their missing values to be inferred through geocoding.
* *issue_date* and *license_expiration_date* should be parsed as datetime objects on import.
* All address columns should be combined into one column containing the full address.
* Columns will be converted to more appropriate data types such as float, integer, and category.

### 2.2 Processing Missing Data & Data Types

***Overview:***
* Update the data types of columns listed above
* Split *latitude_longitude* into separate columns and convert to float values: *latitude*, *longitude*<br>
* Combine address columns into one columns: *full_address*<br>
* Geocode missing *latitude_longitude* with *full_address*<br>
* Geocode missing *zip_code* with complete *latitude_longitude*<br>
* Geocode any missing *full_address* with *latitude_longitude*<br>